In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
#Create spark session
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]
spark = SparkSession.builder \
    .appName("SparkByExamples.com") \
    .getOrCreate()
columns= ["Product","Amount","Country"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

24/12/20 03:11:44 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/12/20 03:11:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/20 03:11:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/20 03:11:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/20 03:11:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


root
 |-- Product: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- Country: string (nullable = true)



+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
|Banana |1000  |USA    |
|Carrots|1500  |USA    |
|Beans  |1600  |USA    |
|Orange |2000  |USA    |
|Orange |2000  |USA    |
|Banana |400   |China  |
|Carrots|1200  |China  |
|Beans  |1500  |China  |
|Orange |4000  |China  |
|Banana |2000  |Canada |
|Carrots|2000  |Canada |
|Beans  |2000  |Mexico |
+-------+------+-------+



In [5]:
pivotDF = df.groupBy('Country').pivot('Product').sum('Amount')
pivotDF.show()

+-------+------+-----+-------+------+
|Country|Banana|Beans|Carrots|Orange|
+-------+------+-----+-------+------+
|  China|   400| 1500|   1200|  4000|
|    USA|  1000| 1600|   1500|  4000|
| Mexico|  NULL| 2000|   NULL|  NULL|
| Canada|  2000| NULL|   2000|  NULL|
+-------+------+-----+-------+------+



In [8]:
pivotDF = df.groupBy("Product","Country") \
      .sum("Amount") \
      .groupBy("Product") \
      .pivot("Country") \
      .sum("sum(Amount)") 
pivotDF.show(truncate=False)

+-------+-------+-----------+
|Product|Country|sum(Amount)|
+-------+-------+-----------+
|Banana |USA    |1000       |
|Carrots|USA    |1500       |
|Beans  |USA    |1600       |
|Orange |USA    |4000       |
|Banana |China  |400        |
|Carrots|China  |1200       |
|Orange |China  |4000       |
|Beans  |China  |1500       |
|Banana |Canada |2000       |
|Carrots|Canada |2000       |
|Beans  |Mexico |2000       |
+-------+-------+-----------+



In [10]:
from pyspark.sql.functions import expr
unpivotExpr = "stack(3, 'Canada', Canada, 'China', China, 'Mexico', Mexico) as (Country,Total)"
unPivotDF = pivotDF.select("Product", expr(unpivotExpr)).where("Total is not null")
unPivotDF.show(truncate=False)
unPivotDF.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Canada` cannot be resolved. Did you mean one of the following? [`Country`, `Product`, `sum(Amount)`].; line 1 pos 19;
'Project [Product#0, 'stack(3, Canada, 'Canada, China, 'China, Mexico, 'Mexico) AS (Country, Total)]
+- Aggregate [Product#0, Country#2], [Product#0, Country#2, sum(Amount#1L) AS sum(Amount)#330L]
   +- LogicalRDD [Product#0, Amount#1L, Country#2], false
